In [ ]:
from SOAPpy import WSDL
#can't work in 3.10 because of version
#pip install SOAPpy, version

# Do a request on Worms to get species data

URL for the example : https://www.marinespecies.org/rest/AphiaRecordByAphiaID/138675
 

In [ ]:
import json
import pandas as pd
import requests

#Init a dataframe
df = []

# Get a record for one APHIAID
url = "https://www.marinespecies.org/rest/AphiaRecordByAphiaID/138675"
r = requests.get(url)
#print(r.text)
print(type(r.json()))

data = r.json()
#print(data) 

# Format the record into one line
row = []
for e in data.keys():
    #print(e)
    #print(data[e])
    row.append(data[e])

# Add the line into the dataframe
df.append(row)

#Finalize the dataframe with column names
resultat = pd.DataFrame(df, columns=data.keys())

resultat


## Renseigner tout le fichier excel

In [ ]:
import json 
import requests
import pandas as pd


In [ ]:
# Fonction pour avoir les données worms 
def get_worms_data(aphiaid):
    url = f"https://www.marinespecies.org/rest/AphiaRecordByAphiaID/{aphiaid}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        return None

In [ ]:
#https://www.marinespecies.org/aphia.php?p=taxdetails&id=137092
data = get_worms_data(137092)

type(data)
nom_colonnes = data.keys()

print(nom_colonnes)

data

In [ ]:
## Lire le fichier Excel


# Lire le fichier Excel
excel_file = "C:\Travail\Enseignement\Cours_M2_python\\2023\data\Table_espece_UTF8_simplifie.xlsx"
sheet_name = "Espece_incomplet"
df = pd.read_excel(excel_file, sheet_name)

print(df.columns)
df


In [ ]:
# Parcourir les lignes
for index, row in df.iterrows():
    aphiaid = row['aphiaid_accepted']  # Colonne du Excel avec les AphiaID
    worms_data = get_worms_data(aphiaid)
    print(aphiaid)
    if worms_data:
        # Parcourir dynamiquement toutes les colonnes du DataFrame
        for column in df.columns:
            for k in worms_data.keys():
                if column.lower() == k.lower() : 
                    print('\t column {0} et data.k {1}'.format(column, k))
                    df.at[index, column] = worms_data[k]
        



### Complement par rapport au cours

Certaines colonnes ne correspondent pas tout à fait mais peuvent être renseignées : 
- ScientificName_accepted
- Authority_accepted
- taxonRank
- taxon_APHIAID

In [ ]:
# Parcourir les lignes
for index, row in df.iterrows():
    aphiaid = row['aphiaid_accepted']  # Colonne du Excel avec les AphiaID
    worms_data = get_worms_data(aphiaid)
    print(aphiaid)
    if worms_data:
        # Parcourir dynamiquement toutes les colonnes du DataFrame
        for column in df.columns:
            for k in worms_data.keys():
                if column.lower() == k.lower() : 
                    print('\t column {0} et data.k {1}'.format(column, k))
                    df.at[index, column] = worms_data[k]
        #Certaines colonnes ne correspondent pas tout à fait mais peuvent être renseignées (sauf Subgenus	Species	Subspecies)
        ##ScientificName_accepted	Authority_accepted 	taxonRank
        df.at[index, 'ScientificName_accepted'] = worms_data['scientificname']
        df.at[index, 'Authority_accepted'] = worms_data['valid_authority']
        df.at[index, 'taxonRank'] = worms_data['rank']
        df.at[index, 'taxon_APHIAID']= worms_data['valid_AphiaID']
        


In [ ]:
df

## Récupérer les noms communs (vernaculaires)

https://www.marinespecies.org/rest/AphiaVernacularsByAphiaID/{ID}

Documentation REST [ici](https://www.marinespecies.org/rest/)

In [ ]:
# Fonction pour avoir les données worms 
def get_worms_vernaculars(aphiaid):
    url = f"https://www.marinespecies.org/rest/AphiaVernacularsByAphiaID/{aphiaid}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        return None

In [ ]:
test = get_worms_vernaculars(137092)

# On récupère une liste de {'vernacular': 'xxx', 'language_code': 'yyy', 'language': 'zzz'}
test

In [ ]:
for e in test:
    if e['language_code'] == 'fra' or  e['language_code'] == 'eng' :
        print(e['vernacular']) 

In [ ]:
# Parcourir les lignes
for index, row in df.iterrows():
    aphiaid = row['aphiaid_accepted']  # Colonne du Excel avec les AphiaID
    worms_data = get_worms_vernaculars(aphiaid)
    print(aphiaid)
    if worms_data:
        # Parcourir dynamiquement toutes les colonnes du DataFrame
        # Renseigner nom_commun_fr	nom_commun_en
        for e in worms_data:
            if e['language_code'] == 'fra'  :
                print(e['vernacular']) 
                df.at[index, 'nom_commun_fr'] = e['vernacular']
            elif e['language_code'] == 'eng' :
                df.at[index, 'nom_commun_en'] = e['vernacular']
                    


In [ ]:
df

## Sauver les résultats

In [ ]:
# Enregistrer les modifications dans le fichier Excel d'origine
new_excel_file = "C:\Travail\Enseignement\Cours_M2_python\\2023\data\Table_espece_UTF8_simplifie_complet.xlsx"
df.to_excel(new_excel_file, sheet_name=sheet_name, index=False)